This is an example notebook of how to use the NIRVAR package to simulate from the NIRVAR model and do NIRVAR estimation.

In [1]:
# Required packages 
import numpy as np 
from numpy.random import default_rng
from src.models import generativeVAR
from src.models import train_model 
import plotly.express as px
import plotly.graph_objects as go


In [2]:
###### HYPERPARAMETERS ###### 
SEED = 352
random_state = default_rng(seed=SEED) 
N = 100
T=1000
Q=1
B=5 # number of blocks
n_iter = 7 
spectral_radius = 0.9
p_in = 1 # probability of connection within a block
p_out = 0.03 # probability of connection between blocks

In [3]:
###### SIMULATE DATA ###### 
W = np.random.uniform(low=0.5,high=1,size=(N, N))
generator = generativeVAR.generativeVAR(random_state=random_state,
                                        T=T,
                                        B=B,
                                        N=N,
                                        Q=Q,
                                        multiplier=spectral_radius,
                                        p_in=p_in,
                                        p_out=p_out,
                                        phi_distribution=W
                                        )
X = generator.generate()


In [4]:
# Visualise generated time series 
components_to_plot = [97,98]
timelength_to_plot = 100
fig = go.Figure()
for i in components_to_plot:
    fig.add_trace(go.Scatter(x=np.arange(timelength_to_plot), y=X[:timelength_to_plot,i,0], mode='lines', name=f'Component {i+1}'))

fig.update_layout(
                  xaxis_title='T',
                  yaxis_title=r'$X_{T}$')   

fig.show()

In [5]:
# Plot A 
fig = px.imshow(generator.adjacency_matrix[:,0,:,0],zmin=-1,zmax=1,color_continuous_scale='RdBu')
fig.show()

In [6]:
# Plot Phi 
max_phi = np.max(generator.phi_coefficients[:,0,:,0])
fig = px.imshow(generator.phi_coefficients[:,0,:,0],zmin=-max_phi,zmax=max_phi,color_continuous_scale='RdBu')
fig.show()

In [7]:
###### ESTIMATION ######
# Embed each panel component in a latent space 
current_embedding = train_model.Embedding(X,cutoff_feature=0,embedding_method="Pearson Correlation") 
current_corr = current_embedding.pearson_correlations()
current_embedded_array = current_embedding.embed_corr_matrix(current_corr,n_iter=n_iter,random_state=235)
print(f"Embedding dimension estimated using Marcenko-Pastur cutoff: {current_embedding.d}") 

Embedding dimension estimated using Marcenko-Pastur cutoff: 5


In [8]:
# Plot the embedded points coloured by the ground truth block memberships 
colors = [
    'rgb(55, 126, 184)',   # Plotly Blue
    'rgb(228, 26, 28)',    # Plotly Red
    'rgb(77, 175, 74)',    # Plotly Green
    'rgb(152, 78, 163)',   # Plotly Purple
    'rgb(255, 127, 0)',    # Plotly Orange
    'rgb(0, 139, 139)' ,     # Dark Cyan
    '#e377c2',  # Plotly Pink
]
x_direction = 1
y_direction = 2
fig = go.Figure()
for i in range(B):
    fig.add_trace(go.Scatter(x=current_embedded_array[0,int(i*N/B):int((i+1)*N/B),x_direction], y=current_embedded_array[0,int(i*N/B):int((i+1)*N/B),y_direction], mode='markers', marker=dict(color=colors[i]),name=f'Block {i+1}'))
# fig.add_trace(go.Scatter(x=current_embedded_array[0,:20,1], y=current_embedded_array[0,:20,2], mode='markers', marker=dict(color=colors[0]),name='Block 1'))
# fig.add_trace(go.Scatter(x=current_embedded_array[0,20:40,1], y=current_embedded_array[0,20:40,2], mode='markers', marker=dict(color=colors[1]),name='Block 2'))
# fig.add_trace(go.Scatter(x=current_embedded_array[0,40:60,1], y=current_embedded_array[0,40:60,2], mode='markers', marker=dict(color=colors[2]),name='Block 3'))
# fig.add_trace(go.Scatter(x=current_embedded_array[0,60:80,1], y=current_embedded_array[0,60:80,2], mode='markers', marker=dict(color=colors[3]),name='Block 4'))
# fig.add_trace(go.Scatter(x=current_embedded_array[0,80:,1], y=current_embedded_array[0,80:,2], mode='markers', marker=dict(color=colors[4]),name='Block 5'))
layout = go.Layout(
    yaxis=dict(title='Third principle direction', showline=True, linewidth=1, linecolor='black',ticks='outside',mirror=True),
    xaxis=dict(title='Second principle direction',showline=True, linewidth=1, linecolor='black',ticks='outside',mirror=True,automargin=True),
    paper_bgcolor='white',  # Set background color to white
    plot_bgcolor='white',   # Set plot area color to white
    font_family="Serif", 
    font_size=16, 
    margin=dict(l=5, r=5, t=5, b=5),
    width =500, 
    height=350
)
fig.update_layout(layout)
fig.show()

In [9]:
# Cluster the components using a Gaussian Mixture Model 
trainer = train_model.fit(current_embedded_array,X,target_feature=0)
neighbours , labels = trainer.gmm(k=current_embedding.d) 

In [10]:
print(labels)

[[3. 3. 1. 2. 2. 2. 2. 2. 3. 2. 2. 2. 2. 3. 2. 2. 4. 3. 3. 2. 0. 0. 3. 0.
  0. 0. 0. 0. 0. 0. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 3. 3. 3. 3. 3. 4. 3. 3.
  2. 3. 3. 3. 3. 3. 3. 4. 2. 3. 3. 3. 3. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 4. 4. 4. 4. 4. 4. 4. 3. 4. 4. 4. 4. 4. 4. 4. 4.
  4. 3. 1. 4.]]


In [11]:
# Plot the embedded points coloured by the Gaussian Mixture Model cluster labels 
fig = go.Figure()
for i in range(B):
    fig.add_trace(go.Scatter(x=current_embedded_array[0,labels[0]==i,x_direction], y=current_embedded_array[0,labels[0]==i,y_direction], mode='markers', marker=dict(color=colors[i]),name=f'Cluster {i+1}'))
layout = go.Layout(
    yaxis=dict(title='Third principle direction', showline=True, linewidth=1, linecolor='black',ticks='outside',mirror=True),
    xaxis=dict(title='Second principle direction',showline=True, linewidth=1, linecolor='black',ticks='outside',mirror=True,automargin=True),
    paper_bgcolor='white',  # Set background color to white
    plot_bgcolor='white',   # Set plot area color to white
    font_family="Serif", 
    font_size=16, 
    margin=dict(l=5, r=5, t=5, b=5),
    width =500, 
    height=350
)
fig.update_layout(layout)
fig.show()

In [12]:
# Do OLS estimation of unrestricted parameters 
ols_params = trainer.ols_parameters(neighbours)
phi_hat = ols_params[:,:,0] 

In [13]:
# Plot the estimated VAR coefficient matrix, Phi
max_phi = np.max(phi_hat)
fig = px.imshow(phi_hat,zmin=-max_phi,zmax=max_phi,color_continuous_scale='RdBu')
fig.show()